In [1]:
import configparser
import os
import sys
from time import sleep
from uuid import uuid4

import numpy as np

import GCode
import GRBL

#

# Code:

In [2]:
# Poplar 1x4". Cut
BlockHeight = 68.0  # [mm]
BlockLength = 143  # [mm]
BlockThickness = 19.0  # [mm]
print("{} x {} x {}".format(BlockHeight, BlockLength, BlockThickness))


68.0 x 143 x 19.0


In [3]:
config = configparser.ConfigParser()
config["GRBL"] = dict()
config["GRBL"]["port"] = "/dev/cnc_3018"
cnc = GRBL.GRBL(**config["GRBL"])

cnc.reset()

(7, ['', "Grbl 1.1f ['$' for help]", "[MSG:'$H'|'$X' to unlock]"])

In [7]:
cnc.home()

14

In [4]:
if "Alarm" in cnc.status:
    cnc.home()

In [8]:
cnc.init()

True

In [9]:
cnc.status

'<Idle|MPos:-255.900,-136.000,-1.000|Bf:15,127|FS:0,0|Ov:100,100,100>'

# Generate Program


In [13]:
def dot(X=10, Y=10, Z=0, power=1, dwell=1, **kwargs):
    program = GCode.GCode(**kwargs)
    
    program.G0(X=X, Y=Y, Z=Z)
    program.M3(S=power)
    program.G1(Z=Z, Y=Y, X=X)
    program.G4(P=dwell)
    program.M5()  # Laser. Off.

    return program

def dot_grid(X=(0, 10), Y=(0, 10), Z=0, n=3, power=1, dwell=1, **kwargs):
    XX = np.linspace(X[0], X[1], n, endpoint=True)
    YY = np.linspace(Y[0], Y[1], n, endpoint=True)
    XXX, YYY = np.meshgrid(XX, YY)
    program = GCode.GCode(**kwargs)
    
    for XXX_, YYY_ in zip(XXX, YYY):
        for X, Y in zip(XXX_, YYY_):
            program+=dot(X=X, Y=Y, Z=Z, power=power, dwell=dwell)
    return program

def dot(X=10, Y=10, Z=0, power=1, dwell=1, **kwargs):
    program = GCode.GCode(**kwargs)
    
    program.G0(X=X, Y=Y, Z=Z)
    program.M3(S=power)
    program.G1(Z=Z, Y=Y, X=X)
    program.G4(P=dwell)
    program.M5()  # Laser. Off.

    return program

def dot_grid(XX=(0, 10), YY=(0, 10), Z=0, power=1, dwell=1, **kwargs):
    XXX, YYY = np.meshgrid(XX, YY)
    program = GCode.GCode(**kwargs)
    
    for XXX_, YYY_ in zip(XXX, YYY):
        for X, Y in zip(XXX_, YYY_):
            program+=dot(X=X, Y=Y, Z=Z, power=power, dwell=dwell)
    return program

In [14]:
dot_grid()

<GCode>[cmds=20]

In [34]:
spacings=[0.1, 0.1, 1, 2, 5]

In [35]:
spacings

[0.1, 0.1, 1, 2, 5]

In [38]:
for spacing in spacings:
    break

In [39]:
X0=10
XX=np.arange(X0, X0+5, spacing)
XX

array([10. , 10.1, 10.2, 10.3, 10.4, 10.5, 10.6, 10.7, 10.8, 10.9, 11. ,
       11.1, 11.2, 11.3, 11.4, 11.5, 11.6, 11.7, 11.8, 11.9, 12. , 12.1,
       12.2, 12.3, 12.4, 12.5, 12.6, 12.7, 12.8, 12.9, 13. , 13.1, 13.2,
       13.3, 13.4, 13.5, 13.6, 13.7, 13.8, 13.9, 14. , 14.1, 14.2, 14.3,
       14.4, 14.5, 14.6, 14.7, 14.8, 14.9])

In [40]:
Y0=10
YY=np.arange(Y0, Y0+5, spacing)
YY

array([10. , 10.1, 10.2, 10.3, 10.4, 10.5, 10.6, 10.7, 10.8, 10.9, 11. ,
       11.1, 11.2, 11.3, 11.4, 11.5, 11.6, 11.7, 11.8, 11.9, 12. , 12.1,
       12.2, 12.3, 12.4, 12.5, 12.6, 12.7, 12.8, 12.9, 13. , 13.1, 13.2,
       13.3, 13.4, 13.5, 13.6, 13.7, 13.8, 13.9, 14. , 14.1, 14.2, 14.3,
       14.4, 14.5, 14.6, 14.7, 14.8, 14.9])

In [41]:
program = dot_grid(XX=XX, YY=YY, Z=-14, power=255, dwell=1)

In [42]:
program.machine=cnc

In [43]:
program.run()